# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** 


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** 

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** 

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** 

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 10          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 256           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(encoder.embed.parameters())+list(decoder.parameters()) 

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr=0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=0.88s)
creating index...


  0%|          | 1042/414113 [00:00<01:23, 4927.27it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [01:05<00:00, 6336.30it/s]


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [4]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time
from workspace_utils import active_session


with active_session():

# Open the training log file.
    f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [100/41412], Loss: 4.1101, Perplexity: 60.9527
Epoch [1/3], Step [200/41412], Loss: 4.4056, Perplexity: 81.90824
Epoch [1/3], Step [300/41412], Loss: 3.9829, Perplexity: 53.67076
Epoch [1/3], Step [400/41412], Loss: 3.7059, Perplexity: 40.68806
Epoch [1/3], Step [500/41412], Loss: 3.2541, Perplexity: 25.8975
Epoch [1/3], Step [600/41412], Loss: 3.3208, Perplexity: 27.68147
Epoch [1/3], Step [700/41412], Loss: 4.0262, Perplexity: 56.04952
Epoch [1/3], Step [800/41412], Loss: 4.1236, Perplexity: 61.7799
Epoch [1/3], Step [900/41412], Loss: 3.4994, Perplexity: 33.09622
Epoch [1/3], Step [1000/41412], Loss: 3.4623, Perplexity: 31.8893
Epoch [1/3], Step [1100/41412], Loss: 3.3164, Perplexity: 27.5615
Epoch [1/3], Step [1200/41412], Loss: 3.6987, Perplexity: 40.3953
Epoch [1/3], Step [1300/41412], Loss: 3.9696, Perplexity: 52.9642
Epoch [1/3], Step [1400/41412], Loss: 2.9887, Perplexity: 19.8602
Epoch [1/3], Step [1500/41412], Loss: 3.6121, Perplexity: 37.0451
Epoch [1/3], 

Epoch [1/3], Step [12400/41412], Loss: 3.2267, Perplexity: 25.1963
Epoch [1/3], Step [12500/41412], Loss: 2.7405, Perplexity: 15.4948
Epoch [1/3], Step [12600/41412], Loss: 2.3200, Perplexity: 10.1759
Epoch [1/3], Step [12700/41412], Loss: 2.7628, Perplexity: 15.8449
Epoch [1/3], Step [12800/41412], Loss: 2.6115, Perplexity: 13.6199
Epoch [1/3], Step [12900/41412], Loss: 3.2146, Perplexity: 24.8923
Epoch [1/3], Step [13000/41412], Loss: 2.0399, Perplexity: 7.68998
Epoch [1/3], Step [13100/41412], Loss: 2.3791, Perplexity: 10.7952
Epoch [1/3], Step [13200/41412], Loss: 2.6570, Perplexity: 14.2531
Epoch [1/3], Step [13300/41412], Loss: 3.1434, Perplexity: 23.1820
Epoch [1/3], Step [13400/41412], Loss: 2.1021, Perplexity: 8.18313
Epoch [1/3], Step [13500/41412], Loss: 2.8814, Perplexity: 17.8400
Epoch [1/3], Step [13600/41412], Loss: 2.5676, Perplexity: 13.0342
Epoch [1/3], Step [13700/41412], Loss: 2.5462, Perplexity: 12.7591
Epoch [1/3], Step [13800/41412], Loss: 2.8492, Perplexity: 17.

Epoch [1/3], Step [24600/41412], Loss: 1.4638, Perplexity: 4.32226
Epoch [1/3], Step [24700/41412], Loss: 2.3258, Perplexity: 10.2346
Epoch [1/3], Step [24800/41412], Loss: 1.7633, Perplexity: 5.83161
Epoch [1/3], Step [24900/41412], Loss: 2.8746, Perplexity: 17.7182
Epoch [1/3], Step [25000/41412], Loss: 2.3691, Perplexity: 10.6875
Epoch [1/3], Step [25100/41412], Loss: 2.4099, Perplexity: 11.1328
Epoch [1/3], Step [25200/41412], Loss: 2.4268, Perplexity: 11.3227
Epoch [1/3], Step [25300/41412], Loss: 2.8277, Perplexity: 16.9063
Epoch [1/3], Step [25400/41412], Loss: 2.1472, Perplexity: 8.56059
Epoch [1/3], Step [25500/41412], Loss: 2.6860, Perplexity: 14.6724
Epoch [1/3], Step [25600/41412], Loss: 2.2717, Perplexity: 9.69585
Epoch [1/3], Step [25700/41412], Loss: 2.4952, Perplexity: 12.1246
Epoch [1/3], Step [25800/41412], Loss: 2.8768, Perplexity: 17.7579
Epoch [1/3], Step [25900/41412], Loss: 2.8737, Perplexity: 17.7023
Epoch [1/3], Step [26000/41412], Loss: 3.1510, Perplexity: 23.

Epoch [1/3], Step [36800/41412], Loss: 2.0356, Perplexity: 7.65650
Epoch [1/3], Step [36900/41412], Loss: 2.1624, Perplexity: 8.69228
Epoch [1/3], Step [37000/41412], Loss: 2.1704, Perplexity: 8.76164
Epoch [1/3], Step [37100/41412], Loss: 2.4682, Perplexity: 11.8010
Epoch [1/3], Step [37200/41412], Loss: 2.4146, Perplexity: 11.1853
Epoch [1/3], Step [37300/41412], Loss: 2.3960, Perplexity: 10.9796
Epoch [1/3], Step [37400/41412], Loss: 2.1915, Perplexity: 8.94825
Epoch [1/3], Step [37500/41412], Loss: 1.8753, Perplexity: 6.52296
Epoch [1/3], Step [37600/41412], Loss: 2.5919, Perplexity: 13.3555
Epoch [1/3], Step [37700/41412], Loss: 2.3105, Perplexity: 10.0792
Epoch [1/3], Step [37800/41412], Loss: 1.9145, Perplexity: 6.78355
Epoch [1/3], Step [37900/41412], Loss: 2.2642, Perplexity: 9.62378
Epoch [1/3], Step [38000/41412], Loss: 2.9217, Perplexity: 18.5725
Epoch [1/3], Step [38100/41412], Loss: 2.1847, Perplexity: 8.88794
Epoch [1/3], Step [38200/41412], Loss: 2.1492, Perplexity: 8.5

Epoch [2/3], Step [7700/41412], Loss: 1.8756, Perplexity: 6.52478
Epoch [2/3], Step [7800/41412], Loss: 2.3996, Perplexity: 11.0190
Epoch [2/3], Step [7900/41412], Loss: 2.3059, Perplexity: 10.0337
Epoch [2/3], Step [8000/41412], Loss: 2.0283, Perplexity: 7.60142
Epoch [2/3], Step [8100/41412], Loss: 2.6342, Perplexity: 13.9315
Epoch [2/3], Step [8200/41412], Loss: 2.2367, Perplexity: 9.36201
Epoch [2/3], Step [8300/41412], Loss: 2.3081, Perplexity: 10.0548
Epoch [2/3], Step [8400/41412], Loss: 2.7772, Perplexity: 16.0746
Epoch [2/3], Step [8500/41412], Loss: 1.9147, Perplexity: 6.78493
Epoch [2/3], Step [8600/41412], Loss: 2.3370, Perplexity: 10.3505
Epoch [2/3], Step [8700/41412], Loss: 2.2068, Perplexity: 9.08694
Epoch [2/3], Step [8800/41412], Loss: 2.3211, Perplexity: 10.1869
Epoch [2/3], Step [8900/41412], Loss: 2.4454, Perplexity: 11.5356
Epoch [2/3], Step [9000/41412], Loss: 2.3753, Perplexity: 10.7540
Epoch [2/3], Step [9100/41412], Loss: 3.0571, Perplexity: 21.2660
Epoch [2/3

...

Epoch [2/3], Step [32100/41412], Loss: 2.2187, Perplexity: 9.19551
Epoch [2/3], Step [32200/41412], Loss: 2.4468, Perplexity: 11.5517
Epoch [2/3], Step [32300/41412], Loss: 1.8178, Perplexity: 6.15828
Epoch [2/3], Step [32400/41412], Loss: 2.9839, Perplexity: 19.7653
Epoch [2/3], Step [32500/41412], Loss: 2.2296, Perplexity: 9.29572
Epoch [2/3], Step [32600/41412], Loss: 2.5647, Perplexity: 12.9963
Epoch [2/3], Step [32700/41412], Loss: 2.7110, Perplexity: 15.0437
Epoch [2/3], Step [32800/41412], Loss: 1.9641, Perplexity: 7.12866
Epoch [2/3], Step [32900/41412], Loss: 2.6968, Perplexity: 14.8325
Epoch [2/3], Step [33000/41412], Loss: 2.2552, Perplexity: 9.53716
Epoch [2/3], Step [33100/41412], Loss: 2.1756, Perplexity: 8.80785
Epoch [2/3], Step [33200/41412], Loss: 1.9340, Perplexity: 6.91696
Epoch [2/3], Step [33300/41412], Loss: 2.6890, Perplexity: 14.7171
Epoch [2/3], Step [33400/41412], Loss: 2.2302, Perplexity: 9.30214
Epoch [2/3], Step [33500/41412], Loss: 2.3722, Perplexity: 10.

Epoch [3/3], Step [2900/41412], Loss: 2.3042, Perplexity: 10.0159
Epoch [3/3], Step [3000/41412], Loss: 2.3354, Perplexity: 10.3335
Epoch [3/3], Step [3100/41412], Loss: 2.3506, Perplexity: 10.4916
Epoch [3/3], Step [3200/41412], Loss: 2.5531, Perplexity: 12.8464
Epoch [3/3], Step [3300/41412], Loss: 2.0737, Perplexity: 7.95397
Epoch [3/3], Step [3400/41412], Loss: 2.4406, Perplexity: 11.4797
Epoch [3/3], Step [3500/41412], Loss: 2.3657, Perplexity: 10.6513
Epoch [3/3], Step [3600/41412], Loss: 2.2091, Perplexity: 9.10730
Epoch [3/3], Step [3700/41412], Loss: 1.8170, Perplexity: 6.15374
Epoch [3/3], Step [3800/41412], Loss: 2.0686, Perplexity: 7.91392
Epoch [3/3], Step [3900/41412], Loss: 2.5632, Perplexity: 12.9769
Epoch [3/3], Step [4000/41412], Loss: 1.8842, Perplexity: 6.58115
Epoch [3/3], Step [4100/41412], Loss: 2.0249, Perplexity: 7.57507
Epoch [3/3], Step [4200/41412], Loss: 1.9806, Perplexity: 7.24702
Epoch [3/3], Step [4300/41412], Loss: 2.5231, Perplexity: 12.4673
Epoch [3/3

Epoch [3/3], Step [15200/41412], Loss: 2.5274, Perplexity: 12.5205
Epoch [3/3], Step [15300/41412], Loss: 2.0735, Perplexity: 7.95255
Epoch [3/3], Step [15400/41412], Loss: 1.9697, Perplexity: 7.16869
Epoch [3/3], Step [15500/41412], Loss: 2.2015, Perplexity: 9.03868
Epoch [3/3], Step [15600/41412], Loss: 2.0370, Perplexity: 7.66758
Epoch [3/3], Step [15700/41412], Loss: 2.6438, Perplexity: 14.0665
Epoch [3/3], Step [15800/41412], Loss: 2.1839, Perplexity: 8.88050
Epoch [3/3], Step [15900/41412], Loss: 2.3885, Perplexity: 10.8971
Epoch [3/3], Step [16000/41412], Loss: 1.9395, Perplexity: 6.95556
Epoch [3/3], Step [16100/41412], Loss: 2.2560, Perplexity: 9.54461
Epoch [3/3], Step [16200/41412], Loss: 1.6659, Perplexity: 5.29050
Epoch [3/3], Step [16300/41412], Loss: 2.6780, Perplexity: 14.5555
Epoch [3/3], Step [16400/41412], Loss: 2.2983, Perplexity: 9.95720
Epoch [3/3], Step [16500/41412], Loss: 1.9018, Perplexity: 6.69806
Epoch [3/3], Step [16600/41412], Loss: 2.2835, Perplexity: 9.8

Epoch [3/3], Step [27400/41412], Loss: 1.8046, Perplexity: 6.07769
Epoch [3/3], Step [27500/41412], Loss: 2.1933, Perplexity: 8.96454
Epoch [3/3], Step [27600/41412], Loss: 2.0592, Perplexity: 7.83988
Epoch [3/3], Step [27700/41412], Loss: 2.1914, Perplexity: 8.94807
Epoch [3/3], Step [27800/41412], Loss: 2.0452, Perplexity: 7.73099
Epoch [3/3], Step [27900/41412], Loss: 2.0829, Perplexity: 8.02744
Epoch [3/3], Step [28000/41412], Loss: 2.2732, Perplexity: 9.71002
Epoch [3/3], Step [28100/41412], Loss: 2.2705, Perplexity: 9.68424
Epoch [3/3], Step [28200/41412], Loss: 1.8850, Perplexity: 6.58665
Epoch [3/3], Step [28300/41412], Loss: 1.8869, Perplexity: 6.59905
Epoch [3/3], Step [28400/41412], Loss: 1.8704, Perplexity: 6.49072
Epoch [3/3], Step [28500/41412], Loss: 2.3384, Perplexity: 10.3644
Epoch [3/3], Step [28600/41412], Loss: 2.0671, Perplexity: 7.90227
Epoch [3/3], Step [28700/41412], Loss: 2.8758, Perplexity: 17.7388
Epoch [3/3], Step [28800/41412], Loss: 2.1731, Perplexity: 8.7

Epoch [3/3], Step [39600/41412], Loss: 2.0497, Perplexity: 7.76577
Epoch [3/3], Step [39700/41412], Loss: 2.0691, Perplexity: 7.91765
Epoch [3/3], Step [39800/41412], Loss: 1.9019, Perplexity: 6.69898
Epoch [3/3], Step [39900/41412], Loss: 2.3449, Perplexity: 10.4326
Epoch [3/3], Step [40000/41412], Loss: 1.9205, Perplexity: 6.82446
Epoch [3/3], Step [40100/41412], Loss: 2.2055, Perplexity: 9.07489
Epoch [3/3], Step [40200/41412], Loss: 2.1693, Perplexity: 8.75248
Epoch [3/3], Step [40300/41412], Loss: 2.5315, Perplexity: 12.5724
Epoch [3/3], Step [40400/41412], Loss: 2.2750, Perplexity: 9.72827
Epoch [3/3], Step [40500/41412], Loss: 1.8312, Perplexity: 6.24121
Epoch [3/3], Step [40600/41412], Loss: 1.9426, Perplexity: 6.97683
Epoch [3/3], Step [40700/41412], Loss: 2.2863, Perplexity: 9.83854
Epoch [3/3], Step [40800/41412], Loss: 2.5843, Perplexity: 13.2539
Epoch [3/3], Step [40900/41412], Loss: 2.2258, Perplexity: 9.26122
Epoch [3/3], Step [41000/41412], Loss: 1.7707, Perplexity: 5.8

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.